In [1]:
import time
from random import random as rd
from selenium import webdriver
from bs4 import BeautifulSoup 
import requests
from PIL import Image
import shutil
import pandas as pd
import re
import os
from datetime import date as dt
from selenium.webdriver.chrome.options import Options

/Users/sh-tatsuno/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/sh-tatsuno/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# 日付
str(dt.today())

'2018-08-14'

In [3]:
options = Options()
# ヘッドレスモードを有効にする（次の行をコメントアウトすると画面が表示される）。
options.add_argument('--headless')

In [4]:
def nsleep(t=0.1):
    time.sleep(t+rd()/10)
    return

In [5]:
def download_img(url, file_name):
    r = requests.get(url)
    if r.status_code == 200:
        with open(file_name, 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f)

In [6]:
HOME_DIR = 'https://www.low-ya.com'

## カテゴリ抽出

In [7]:
driver = webdriver.Chrome(chrome_options=options)

# access
# get category from left tab
driver.get('https://www.low-ya.com/category/SOFA/')

#  for 1 page
data = driver.page_source
html = BeautifulSoup(data, "lxml")

l_category = list(map(lambda x: x.string.replace('\n','').replace('\t',''), html.find_all("dt", attrs={"class": "left-category__triger"})))

m_category=[]
for i in range(len(l_category)):
    medium_target = html.find_all("dl", attrs={"class": "left-category-medium"})[i].find_all('dd')
    for cell in medium_target:
        if cell.get('class') == None: # remove noise
            m_category.append([l_category[i], cell.find('a').get('href'), cell.string])
            
# カテゴリ抽出
categorydf = pd.DataFrame(m_category,columns=['large_category','url','medium_category'])

# ALL を除外
categorydf = categorydf[categorydf.medium_category!='ALL'].reset_index(drop=True)

driver.close()

In [8]:
categorydf.head()

,large_category,url,medium_category
0,ソファ,https://www.low-ya.com/category/SOFA_FLOOR/,フロアソファ・ローソファ
1,ソファ,https://www.low-ya.com/category/SOFA_BED/,ソファベッド
2,ソファ,https://www.low-ya.com/category/SOFA_COUCH/,カウチソファ・コーナーソファ
3,ソファ,https://www.low-ya.com/category/SOFA_RECLINING/,リクライニングソファ
4,ソファ,https://www.low-ya.com/category/SOFA_COMPACT/,コンパクトソファ


In [9]:
categorydf.iloc[0]

large_category                                             ソファ
url                https://www.low-ya.com/category/SOFA_FLOOR/
medium_category                                   フロアソファ・ローソファ
Name: 0, dtype: object

## カテゴリごとの家具情報の取得

In [10]:
# retry max
retry_max=5

# all data
item_data = []

# site name
site = 'LAWYA'

# date
datetime = dt.today()

driver = webdriver.Chrome(chrome_options=options)

for i in range(len(categorydf)):
    
    # カテゴリの指定
    large_category = categorydf.iloc[i].large_category
    url = categorydf.iloc[i].url
    medium_category = categorydf.iloc[i].medium_category
    
    print('category: '+ medium_category)
    
    # check while end
    flag = 1

    # access
    driver.get(url)

    page = 0
    flagcounter = 0
    while(flag):
        
        print("page : " +str(page))
        page +=1
        
        #  for 1 page
        data = driver.page_source
        html = BeautifulSoup(data, "lxml")
        
        try: 
            alinks = html.find_all("div", attrs={"class": "search-list"})[0].find_all("a")

            item_num = 0
            counter = 0
            while(item_num < len(alinks)):
                item = alinks[item_num]

                try:
                    # init
                    data = [site, datetime, large_category, medium_category]

                    # url : 詳細URL
                    data.append(item.get("href"))

                    # title : 画像タイトル
                    data.append(item.find("img")['alt'])

                    # img_url : 画像URL
                    data.append(os.path.join('http://', item.find("img")['data-src'][2:]))

                    # 詳細
                    get_url = requests.get(item.get("href"))
                    item_html = BeautifulSoup(get_url.text, "lxml")

                    # price : 価格
                    data.append(int(item_html.find("tr", attrs={"class": "saleYen"}).find("td", attrs={"class": "yen"}).text.replace(',','')))

                    # size : サイズ
                    data.append(item_html.find("div", attrs={"class": "wrap-sizeDetail"}).find("dl").find_all('dd')[0].text)

                    # color: カラー
                    data.append(item_html.find("div", attrs={"class": "wrap-sizeDetail"}).find("dl").find_all('dd')[2].text)

                    item_data.append(data)
                    item_num += 1
                    counter = 0

                except: # retry 
                    print('error')
                    if counter < retry_max:
                        nsleep(2)  # hyper parameter for waiting
                        counter += 1
                    else: # skip
                        print('skip')
                        counter = 0
                        item_num += 1


            # 次に進む
            if(html.find("li", attrs={"class": "search-exist__next"})) != None:
                driver.find_element_by_class_name('search-exist__next').click()  
                nsleep(2) # hyper parameter for waiting
            else:
                flag = 0
                
        except:
            if flagcounter < retry_max:
                nsleep(2)  # hyper parameter for waiting
                flagcounter +=1
            else:
                print('skip category')
                flagcounter = 0
                flag=0

driver.close()


category: フロアソファ・ローソファ
page : 0
page : 1
category: ソファベッド
page : 0
category: カウチソファ・コーナーソファ
page : 0
page : 1
category: リクライニングソファ
page : 0
category: コンパクトソファ
page : 0
page : 1
category: クッションソファ
page : 0
category: スツール・オットマン
page : 0
category: 革・レザー・合皮ソファ
page : 0
page : 1
category: 布張り・ファブリックソファ
page : 0
page : 1
page : 2
page : 3
category: 1人掛けソファ
page : 0
category: 2人掛けソファ
page : 0
page : 1
category: 3人掛けソファ
page : 0
page : 1
category: 4人掛け以上のソファ
page : 0
category: ソファカバー
page : 0
category: コーナーテレビ台
page : 0
category: 伸縮テレビ台
page : 0
category: 壁面収納・ハイタイプテレビ台
page : 0
category: ミドルタイプテレビ台
page : 0
category: ロータイプテレビ台
page : 0
error
page : 1
page : 2
category: 国産テレビ台
page : 0
page : 1
page : 2
error
category: 木製テレビ台
page : 0
page : 1
page : 2
page : 3
page : 4
category: サイドチェスト
page : 0
category: 幅90㎝以下
page : 0
category: 幅91～120㎝
page : 0
category: 幅121～150㎝
page : 0
page : 1
category: 幅151～180㎝
page : 0
page : 1
category: 幅181～210㎝
page : 0
category: 幅211㎝以上
page : 0
category: 食器棚


In [11]:
df = pd.DataFrame(item_data, columns = ['site', 'datetime', 'l_category', 'm_category', 'url', 'title', 'img_url', 'price', 'size', 'color'])

In [12]:
df.head()

,site,datetime,l_category,m_category,url,title,img_url,price,size,color
0,LAWYA,2018-08-14,ソファ,フロアソファ・ローソファ,https://www.low-ya.com/category/SOFA_FLOOR/F20...,2人掛けコンパクトソファ 天然木脚 クッション付き［幅159］,http://dbcn1bdvswqbx.cloudfront.net/client_inf...,30990,幅159ｘ奥行73ｘ高さ68（座面高39）cm,グリーン、ダークブラウン、ベージュ、レッド、グレー、ネイビー、ブラック、デニム
1,LAWYA,2018-08-14,ソファ,フロアソファ・ローソファ,https://www.low-ya.com/category/SOFA_FLOOR/F20...,3人掛けソファ 低く広く美しいロータイプ［幅188］,http://dbcn1bdvswqbx.cloudfront.net/client_inf...,29990,幅188×奥行78×高さ63cm,ブラック、ダークブラウン、キャメル
2,LAWYA,2018-08-14,ソファ,フロアソファ・ローソファ,https://www.low-ya.com/category/SOFA_FLOOR/F20...,3人掛けカウチソファ ラメ入りファブリック ヘッドレスト オットマン付き 全面カバーリング［...,http://dbcn1bdvswqbx.cloudfront.net/client_inf...,49990,幅202ｘ奥行85～200ｘ高77（座面高41）cm,グレー、ダークブラウン、ブラック、グリーン、ベージュ、レッド(茶色みがかった色味となります)...
3,LAWYA,2018-08-14,ソファ,フロアソファ・ローソファ,https://www.low-ya.com/category/SOFA_FLOOR/F20...,3人掛けカウチソファ ヘッドレスト クッション付［幅190］,http://dbcn1bdvswqbx.cloudfront.net/client_inf...,39990,幅190x奥行85～135x高さ56～76cm 座面高さ41cm,ブラウン、グレー、シーダーグリーン、ダークブラウン、ベージュ、ネイビー、チャコールグレー、ブ...
4,LAWYA,2018-08-14,ソファ,フロアソファ・ローソファ,https://www.low-ya.com/category/SOFA_FLOOR/F20...,2人掛け天然木脚ソファ［幅159］,http://dbcn1bdvswqbx.cloudfront.net/client_inf...,29990,幅159x奥行76.5x高さ72cm(座面高さ：39cm),インディゴブルー、グレー、ネイビー、アイボリー、グリーン


In [13]:
df.shape

(3889, 10)

In [14]:
df.to_csv('lawya.csv',index=None)

In [15]:
import sys
sys.setrecursionlimit(100000000)

In [16]:
df.to_pickle('lawya.pkl')

## sandbox

In [ ]:
# for 1 item
for url in map(lambda x: x.get("href"), alinks):
    
    print(folder_name)
    get_url = requests.get(url)
    nsleep(10)
    item_html = BeautifulSoup( \
        get_url.text, "lxml")
    print(item_html)

    item_html.find("h1", attrs={"class": "cartName"}).text
    item_html.find("tr", attrs={"class": "saleYen"}).find("td", attrs={"class": "yen"}).text
    item_html.find("div", {"id":"show"}).extract()
    
    # for all images in 1 item
    links = item_html.find_all("div", attrs={"class": "img_box"})
    
    for link in map(lambda x: x.find('img').get("src").replace('\r','').replace('\n',''), links):
        print(link)
        download_img(HOME_DIR + link, folder_name + '/' + link.split('/')[-1])


# to next page
driver.find_element_by_class_name('search-exist__next').click()

In [79]:
# html 
bs_html = BeautifulSoup( \
    requests.get('https://www.low-ya.com/category/SOFA/F201_G1266.html').text, "lxml")

In [61]:
# 試しに1ファイルダウンロード
r = HOME_DIR + '/client_info/LOWYA/itemimage/F201_G1266_0020V1/color07.jpg'
download_img(r, 'sample.jpg')

In [138]:
# linkの抽出
links = bs_html.find_all("div", attrs={"class": "img_box"})
list(map(lambda x: x.find('img').get("src").replace('\r','').replace('\n',''), links))

['/client_info/LOWYA/itemimage/F201_G1266_0020G1/color03.jpg',
 '/client_info/LOWYA/itemimage/F201_G1266/main01.jpg',
 '/client_info/LOWYA/itemimage/F201_G1266/main02.jpg',
 '/client_info/LOWYA/itemimage/F201_G1266/main03.jpg',
 '/client_info/LOWYA/itemimage/F201_G1266/main04.jpg',
 '/client_info/LOWYA/itemimage/F201_G1266/main05.jpg',
 '/client_info/LOWYA/itemimage/F201_G1266_0020B1/color01.jpg',
 '/client_info/LOWYA/itemimage/F201_G1266_0020F1/color02.jpg',
 '/client_info/LOWYA/itemimage/F201_G1266_0020G1/color03.jpg',
 '/client_info/LOWYA/itemimage/F201_G1266_0020J1/color04.jpg',
 '/client_info/LOWYA/itemimage/F201_G1266_0020M1/color05.jpg',
 '/client_info/LOWYA/itemimage/F201_G1266_0020R1/color06.jpg',
 '/client_info/LOWYA/itemimage/F201_G1266_0020V1/color07.jpg',
 '/client_info/LOWYA/itemimage/F201_G1266_2000AU/color01.jpg']

In [148]:
# \n, \rを除く必要がある
for link in map(lambda x: x.find('img').get("src").replace('\r','').replace('\n',''), links):
    print(HOME_DIR + link, link.split('/')[-1])
    download_img(HOME_DIR + link, link.split('/')[-1])

https://www.low-ya.com/client_info/LOWYA/itemimage/F201_G1266_0020G1/color03.jpg color03.jpg
https://www.low-ya.com/client_info/LOWYA/itemimage/F201_G1266/main01.jpg main01.jpg
https://www.low-ya.com/client_info/LOWYA/itemimage/F201_G1266/main02.jpg main02.jpg
https://www.low-ya.com/client_info/LOWYA/itemimage/F201_G1266/main03.jpg main03.jpg
https://www.low-ya.com/client_info/LOWYA/itemimage/F201_G1266/main04.jpg main04.jpg
https://www.low-ya.com/client_info/LOWYA/itemimage/F201_G1266/main05.jpg main05.jpg
https://www.low-ya.com/client_info/LOWYA/itemimage/F201_G1266_0020B1/color01.jpg color01.jpg
https://www.low-ya.com/client_info/LOWYA/itemimage/F201_G1266_0020F1/color02.jpg color02.jpg
https://www.low-ya.com/client_info/LOWYA/itemimage/F201_G1266_0020G1/color03.jpg color03.jpg
https://www.low-ya.com/client_info/LOWYA/itemimage/F201_G1266_0020J1/color04.jpg color04.jpg
https://www.low-ya.com/client_info/LOWYA/itemimage/F201_G1266_0020M1/color05.jpg color05.jpg
https://www.low-ya.com